In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read the acceleration file in earth's coordinate system which are converted by the watch

In [ ]:
df = pd.DataFrame()
count = 50000

while(True):
    cur = pd.read_csv("../../data/global acc/result" + str(count) + ".csv", skipinitialspace=True, header=None)
    df = df.append(cur, ignore_index=True)
    if count == 500000:
        break
    count += 50000
cur = pd.read_csv("../../data/global acc/result_final.csv", skipinitialspace=True, header=None)
df = df.append(cur, ignore_index=True)

In [ ]:
len(df)

In [ ]:
df.columns = ['global_acc1','global_acc2','global_acc3']

## Add the time, tagname and tester_id into the df

In [ ]:
t = pd.DataFrame.from_csv("../../data/linear_acceleration_grav_mag.csv")
t.columns

In [ ]:
b = t[['tester_id', 'TagName', 'Time']].copy()
for col in ['global_acc1','global_acc2','global_acc3']:
    b[col] = 0

b[['global_acc1','global_acc2','global_acc3']] = df[['global_acc1','global_acc2','global_acc3']].values
b['unix_timestamp'] = pd.to_datetime(b['Time']).values.astype("uint64") / 1000000000
b.to_csv("../../data/global_acc_df.csv")

## Calculate distance and velocity

In [ ]:
from scipy.integrate import simps

In [ ]:
b = pd.DataFrame.from_csv("../../data/global_acc_df.csv")
b.head()

In [ ]:
def correct_velocity_drift(v_0):
    # function to correct velocity drift
    d = v_0[-1] / (len(v_0)-1)
    for i in range(len(v_0)):
        v_0[i] = v_0[i] - d*i
    return v_0

In [ ]:
features = pd.DataFrame(columns=['unix_timestamp','TagName','tester_id',
                                 'v_1','v_2','v_3','d_1','d_2','d_3', 'global_acc1',
                                 'global_acc2','global_acc3','v_12_square','acc_12_square','d_12_square'])


b_groups = b.groupby(['TagName','tester_id'])
count = 0
for key in b_groups.groups.keys():
    
    cur_group = b_groups.get_group(key)
    t = cur_group.unix_timestamp.values
    v_1 = []
    d_1 = []
    v_2 = []
    d_2 = []
    v_3 = []
    d_3 = []
    acc1 = cur_group.global_acc1.values
    acc2 = cur_group.global_acc2.values
    acc3 = cur_group.global_acc3.values
#     Calculate velocity through acceleration
    for i in range(len(t)):
        v_1.append(simps(acc1[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_2.append(simps(acc2[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        v_3.append(simps(acc3[:i+1], x=t[:i+1]))
#         correct velocity drift
    v_1 = correct_velocity_drift(v_1)
    v_2 = correct_velocity_drift(v_2)
    v_3 = correct_velocity_drift(v_3)
#     calculate displacement through velocity
    for i in range(len(t)):
        d_1.append(simps(v_1[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_2.append(simps(v_2[:i+1], x=t[:i+1]))
    for i in range(len(t)):
        d_3.append(simps(v_3[:i+1], x=t[:i+1]))
    
    v_1 = np.array(v_1)
    d_1 = np.array(d_1)
    v_2 = np.array(v_2)
    d_2 = np.array(d_2)
    v_3 = np.array(v_3)
    d_3 = np.array(d_3)

    
    
    temp_df = pd.DataFrame()
    
    temp_df['unix_timestamp'] = t
    temp_df['TagName'] = np.array([key[0]]*len(t))
    temp_df['tester_id'] = np.array([key[1]]*len(t))
                                                           
        
    temp_df['v_1'] = v_1
    temp_df['v_2'] = v_2
    temp_df['v_3'] = v_3
    
    temp_df['d_1'] = d_1                                       
    temp_df['d_2'] = d_2
    temp_df['d_3'] = d_3
    temp_df['global_acc1'] = acc1
    temp_df['global_acc2'] = acc2
    temp_df['global_acc3'] = acc3
    
    temp_df['v_12_square'] = np.sqrt(v_1*v_1 + v_2*v_2)
    temp_df['acc_12_square'] = np.sqrt(acc1*acc1 + acc2*acc2)
    temp_df['d_12_square'] = np.sqrt(d_1*d_1 + d_2*d_2)

    features = features.append(temp_df, ignore_index=True)
    count+=1
    if (count%50==0):
        print(count)

    

In [ ]:
features.head()

In [ ]:
features.to_csv("../../data/global_acc_features_df.csv")